# Proofread_nuclei

In [93]:
# libraries 1
import numpy as np
import pyperclip
import pandas as pd
import pyperclip
from cloudvolume import CloudVolume, view, Bbox
from nglui import statebuilder,annotation,easyviewer,parser
from nglui.statebuilder import *
from nglui.nglite import *
import json

import sys
import os
from datetime import datetime
from caveclient import CAVEclient

sys.path.append(os.path.abspath("../segmentation"))
import authentication_utils as auth
import rootID_lookup as IDlook
sys.path.append(os.path.abspath("../synapses"))
import connectivity_utils

In [94]:
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)

In [95]:
df = pd.read_csv('../Output/soma_info_Aug2021ver5.csv', header=0)

## Create links for proofreading nuclei/somas

Good example: https://github.com/seung-lab/NeuroglancerAnnotationUI/blob/master/examples/statebuilder_examples.ipynb

In [ ]:
# we only need xyz for nuclei/somas..., and make them into lines
xyz_df = df.reindex(columns=['nuc_xyz', 'soma_xyz'])
nuc_xyz_df = df['nuc_xyz'].str.strip('()').str.split(',',expand=True)
soma_xyz_df = df['soma_xyz'].str.strip('()').str.split(',',expand=True)
xyz_df['nuc_xyz'] = nuc_xyz_df.astype(int).values.tolist()
xyz_df['soma_xyz'] = soma_xyz_df.astype(int).values.tolist()

In [ ]:
nuc_xyz_df = nuc_xyz_df.set_axis(['Col_x', 'Col_y', 'Col_z'], axis=1)
sorted_z = nuc_xyz_df.astype(int).sort_values(by=['Col_z'], ascending=True)
sorted_xz = sorted_z.astype(int).sort_values(by=['Col_x'], ascending=True)
sorted_indices = sorted_xz.astype(int).sort_values(by=['Col_y'], ascending=True).index
xyz_df = xyz_df.reindex(sorted_indices).reset_index(drop=True)

In [ ]:
# ..., and make them into lines
lines = LineMapper(point_column_a='nuc_xyz', point_column_b='soma_xyz')

In [ ]:
# make Neuroglancer link
imgTokyo = ImageLayerConfig(name = 'FANCv4-jp',
                                    source = auth.get_cv_path('Image_Tokyo')['url'])
img = ImageLayerConfig(name = 'FANCv4',
                                    source = auth.get_cv_path('Image')['url'])
seg = SegmentationLayerConfig(name = 'seg_Mar2021_proofreading',
                                    source = auth.get_cv_path('FANC_production_segmentation')['url'])     
nuc_Aug = ImageLayerConfig(name = 'nuc',
                                    source = auth.get_cv_path('nuclei_map_Aug2021')['url'])      
nuc_seg_Aug = ImageLayerConfig(name = 'nuc_seg',
                                    source = auth.get_cv_path('nuclei_seg_Aug2021')['url'])        

ann = AnnotationLayerConfig(name='nuc_soma_Aug2021',
                            mapping_rules=lines,
                            tags=['neuron', 'glia','false_positive','soma_need_check'],
                            active = True)

In [ ]:
view_options = {"layout": "xy"}

memory_options = {"gpuMemoryLimit": 4000000000,
                  "systemMemoryLimit": 8000000000,
                  "concurrentDownloads": 64,
                  "jsonStateServer": "https://global.daf-apis.com/nglstate/api/v1/post"}


sb = StateBuilder(layers=[imgTokyo, seg, img, nuc_Aug, nuc_seg_Aug, ann],
                  resolution=[4.3,4.3,45],
                  view_kws=view_options)

In [ ]:
LINK=[]
k=500
minidfs = [xyz_df.loc[i:i+k-1, :] for i in range(0, len(xyz_df), k)]
for dftmp in minidfs:
    # csb = ChainedStateBuilder([sb, vs])                             
    state = json.loads(sb.render_state(dftmp, return_as='json'))
    state.update(memory_options)
    jsn_id = client.state.upload_state_json(state)
    output = client.state.build_neuroglancer_url(jsn_id, auth.get_cv_path('neuroglancer_base')['url'])
    # output =auth.get_cv_path('neuroglancer_base')['url'] + '?json_url={path}{state_id}'.format(path=auth.get_cv_path('json_server')['url'],state_id=client.state.upload_state_json(state))
    LINK.append(output)

In [ ]:
# save into csv
LINK2 = pd.DataFrame(LINK)
LINK2.to_csv('../Output/links_20210903.csv', index=False, header=False)

## Extract tag from proodread csv

functions from https://github.com/seung-lab/NeuroglancerAnnotationUI/blob/36f03cab5ccff8c52b0faba8beff7ab77398ef48/src/nglui/parser/base.py

In [ ]:
# df['is_neuron']=""
# df['is_glia']=""
# df['is_false_positive']=""
# df.to_csv('../Output/proofread_nuc_temp.csv', index=False)

In [96]:
df_progress = pd.read_csv("../Output/proofread_nuc_temp.csv", header=0)
prfrd1 = pd.read_table("../Output/1st_proofreading.tsv", usecols = ['new link'])

In [97]:
rsplitted = prfrd1['new link'].dropna(how='all').str.rsplit('/', 1)
new_id = list()
for i in rsplitted.index:
    new_id.append(rsplitted.loc[i][1])

In [98]:
mylist = []

for i in range(len(new_id)):
    state_id = int(new_id[i])
    state = client.state.get_state_json(state_id)

    # extract info from json state
    nuc_tags = parser.tag_dictionary(state,parser.annotation_layers(state)[0])
    anno_lists = parser.line_annotations(state,parser.annotation_layers(state)[0], tags=True)

    temp = pd.DataFrame({'anno_points_A': anno_lists[0],
                     'anno_points_B': anno_lists[1],
                     'anno_tags': anno_lists[2]})

    # convert [] to [0]
    for j in range(len(temp)):
        if len(temp.iloc[j,2]) == 0:
            temp.iloc[j,2] = [0]
            # temp['anno_tags']

    temp['anno_tags'] = np.concatenate(temp['anno_tags']).astype(int)
    temp['anno_tags'] = temp['anno_tags'].replace(nuc_tags)

    mylist.append(temp)


In [99]:
df_new = pd.concat(mylist).reset_index()

In [100]:
df_new

,index,anno_points_A,anno_points_B,anno_tags
0,0,"[38272, 74224, 1954]","[39072, 73456, 1921]",false_positive
1,1,"[32752, 74304, 1739]","[33056, 73984, 1788]",false_positive
2,2,"[31296, 74672, 1492]","[31200, 74928, 1530]",false_positive
3,3,"[33440, 74896, 1797]","[33968, 74928, 1892]",0
4,4,"[33680, 75136, 1796]","[33744, 75200, 1787]",0
...,...,...,...,...
17071,71,"[42000, 218880, 279]","[42336, 219392, 312]",0
17072,72,"[58736, 219120, 1143]","[58288, 218736, 1101]",false_positive
17073,73,"[46160, 219424, 236]","[46784, 219520, 189]",glia
17074,74,"[53904, 220144, 1319]","[54576, 220320, 1361]",0


In [107]:
print('neuron are {}'.format(sum(df_new['anno_tags']=='neuron')))
print('glia are {}'.format(sum(df_new['anno_tags']=='glia')))
print('false_positive are {}'.format(sum(df_new['anno_tags']=='false_positive')))
print('come_back_to_me_later are {}'.format(sum(df_new['anno_tags']==0)))

neuron are 10499
glia are 1010
false_positive are 389
come_back_to_me_later are 5178


In [111]:
df_new.drop("index", axis=1).to_csv('../Output/1st_proofread_extracted.csv', index=False)

In [112]:
df_new = pd.read_csv("../Output/1st_proofread_extracted.csv", header=0)

In [113]:
df_new

,anno_points_A,anno_points_B,anno_tags
0,"[38272, 74224, 1954]","[39072, 73456, 1921]",false_positive
1,"[32752, 74304, 1739]","[33056, 73984, 1788]",false_positive
2,"[31296, 74672, 1492]","[31200, 74928, 1530]",false_positive
3,"[33440, 74896, 1797]","[33968, 74928, 1892]",0
4,"[33680, 75136, 1796]","[33744, 75200, 1787]",0
...,...,...,...
17071,"[42000, 218880, 279]","[42336, 219392, 312]",0
17072,"[58736, 219120, 1143]","[58288, 218736, 1101]",false_positive
17073,"[46160, 219424, 236]","[46784, 219520, 189]",glia
17074,"[53904, 220144, 1319]","[54576, 220320, 1361]",0


In [120]:
df_new2 = df_new[df_new['anno_tags']=='0']

xyz_df = df_new2.reindex(columns=['anno_points_A', 'anno_points_B'])
nuc_xyz_df = df_new2['anno_points_A'].str.strip('()').str.split(',',expand=True)
soma_xyz_df = df_new2['anno_points_B'].str.strip('()').str.split(',',expand=True)
xyz_df['anno_points_A'] = nuc_xyz_df.astype(int).values.tolist()
xyz_df['anno_points_B'] = soma_xyz_df.astype(int).values.tolist()

ValueError: invalid literal for int() with base 10: '[33440'

In [115]:
xyz_df

,anno_points_A,anno_points_B


In [119]:
df_new[df_new['anno_tags']=='0']

,anno_points_A,anno_points_B,anno_tags
3,"[33440, 74896, 1797]","[33968, 74928, 1892]",0
4,"[33680, 75136, 1796]","[33744, 75200, 1787]",0
28,"[33168, 77808, 1953]","[32992, 78384, 1909]",0
31,"[32528, 78240, 2043]","[33360, 78384, 2028]",0
41,"[32608, 78896, 1959]","[33312, 79008, 1911]",0
...,...,...,...
17066,"[53152, 218432, 139]","[53216, 217600, 235]",0
17067,"[51952, 218496, 401]","[52272, 218560, 373]",0
17068,"[49024, 218576, 516]","[49520, 218560, 486]",0
17071,"[42000, 218880, 279]","[42336, 219392, 312]",0


In [ ]:
for i in range(len(out)):
    segidi = out['Seg ID'].values[i]
    if int(segidi) in progress['latest Seg ID'].values:
        matched = progress.loc[progress['latest Seg ID'] == int(segidi)]
        matched2 = matched.dropna(subset=column_name[3:8],how='all') # remove rows I haven't proofread yet
        matched3 = pd.DataFrame(matched2)
        if len(matched3) > 0:
            last_matched = matched3.iloc[-1,:]
            notes = matched2[column_name[7]].str.cat(sep=', ')
            out.loc[i,column_name[3:8]] = last_matched.loc[column_name[3:8]]
            out.at[out.index[i],column_name[2]] = str(last_matched.saving_pt).strip("''")
            out.at[out.index[i],column_name[7]] = notes